## This notebook is designed to generate an accurate shifting map and determine focal plane of your objects.

In [ ]:
import numpy as np
import OpenLMlib as PSR
import matplotlib.pyplot as plt
from skimage.restoration import inpaint
from image_registration import chi2_shift

### Load LR Images

In [ ]:
file_name_list = list(np.arange(17,22))+list(np.arange(25,30))+list(np.arange(33,38))+list(np.arange(41,46))+list(np.arange(49,54))
frames = []

for i in range(len(file_name_list)):
    path = "D:/0401/30/" + str(file_name_list[i]) + ".dng"
    full_FOV = PSR.largest_FOV(path)
    # use full_FOV[H1:H2, W1:W2] to crop image
    section_FOV = full_FOV # full_FOV[H1:H2, W1:W2]
    frames.append(full_FOV)
    
frames = np.array(frames)

### Generate Shifting Map

A good shifting map ensures that each point is evenly spaced, maintaining consistent distances between neighboring positions.

In [ ]:
shift_map = PSR.shiftMap(frames, len(file_name_list))

plt.rcParams['figure.figsize'] = (6,6)
plt.scatter(shift_map[:,1], shift_map[:,0])
plt.xlabel("Shift [pixel]")
plt.ylabel("Shift [pixel]")
plt.title("Shifts")

### Generate HR Image

If HR image generation fails, it indicates that the shifting map is incorrect. Crop the image to isolate a single target object, then regenerate the shifting map.

In [ ]:
scale_factor = 4

HR_hologram = PSR.superResolution(frames, shift_map, len(file_name_list), scale_factor)

reconstructed_image, ToG, optimized_zs = PSR.imageReconstruction(
    frames[12, :],              
    z_start=450e-6,             
    z_step=10e-6,               
    iteration=20                
)
print(f"Estimated focal plane (LR): {optimized_zs:.6f} m")

# Note: HR focal depth ≈ LR focal depth × (scale_factor²)
reconstructed_image_HR, ToG_HR, optimized_zs_HR = PSR.imageReconstruction(
    HR_hologram,
    z_start=optimized_zs * scale_factor**2 - 100e-6,
    z_step=10e-6,
    iteration=20
)

plt.rcParams['figure.figsize'] = (16, 8)

plt.subplot(1, 2, 1)
plt.imshow(reconstructed_image, cmap="gray")
plt.title("Low Resolution", fontsize=14)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(reconstructed_image_HR, cmap="gray")
plt.title("High Resolution", fontsize=14)
plt.axis("off")

plt.tight_layout()
plt.show()

### Save Shifting Map

Save the shifting map in the same folder as the Windows application and library files

In [ ]:
np.save("path/to/windows_application/shift_map.npy", shift_map)